In [24]:
from chainer.datasets import get_mnist, get_cifar10
from chainer.backends.cuda import to_cpu, to_gpu

import cupy as cp
import numpy as np

In [37]:
train_data, test_data = get_mnist(withlabel=False, ndim=1)

input_dim = 28 * 28

In [41]:
def calc_dim(X):
    u = cp.mean(X, axis=0, keepdims=True)
    C = (X-u) @ (X-u).T
    return cp.trace(C) ** 2 / cp.trace(C @ C)

In [64]:
granule_cell = 'lc'
k = 8
n_hidden = 5000
batch_size = 1000

In [65]:
import model.granule as granule

# seed
np.random.seed(0)
cp.random.seed(0)

# Granule cells
if granule_cell == 'fc':
    gc = granule.FC(m=input_dim, n=n_hidden)
elif granule_cell == 'lc':
    gc = granule.LC(m=input_dim, n=n_hidden, k=k)
elif granule_cell == 'rand':
    gc = granule.Rand(m=input_dim, n=n_hidden, k=k)

In [66]:
idx = np.random.choice(len(train_data), batch_size)
data = train_data[idx]

X = []
for x in data:
    X.append(gc.forward(to_gpu(x)).squeeze(-1))
    
X = cp.array(X)
X.shape

(1000, 5000)

In [67]:
calc_dim(X)

array(29.09468313)